In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
sns.set_context('notebook', font_scale=1.33)
%matplotlib inline

In [3]:
from laverna.task import simulate_reversal_task

simulate_reversal_task([30])

array([[[0, 1, 1],
        [0, 1, 0],
        [0, 1, 0],
        [0, 1, 0],
        [0, 1, 1],
        [1, 1, 0],
        [0, 1, 0],
        [0, 1, 0],
        [0, 1, 0],
        [0, 1, 0],
        [0, 1, 1],
        [0, 0, 0],
        [0, 1, 0],
        [0, 1, 0],
        [0, 1, 0],
        [0, 0, 0],
        [0, 1, 0],
        [0, 1, 0],
        [1, 0, 0],
        [1, 1, 0],
        [0, 1, 0],
        [1, 1, 0],
        [0, 1, 1],
        [1, 0, 0],
        [0, 1, 0],
        [0, 1, 0],
        [0, 1, 0],
        [0, 1, 0],
        [1, 1, 1],
        [0, 1, 0]]])

## Section 1: Noodling
#### 1.1 Exploration on Latent Traits

In [ ]:
np.random.seed(47404)

def inv_logit(arr):
    return 1. / (1 + np.exp(-arr))

x = np.random.normal(0,1,1000)
b = -1
a = 1

theta = inv_logit( a * (x - b) )

sns.distplot(theta, kde=False, bins=np.linspace(0,1,21))

## Section 2: Simmy sims

In [ ]:
from scipy.stats import bernoulli, norm, skewnorm
np.random.seed(0)

## Define metadata.
n_agents = 1000
n_items = 10

## Define simulation parameters.
a = 3
b = 0.5
mu = -1

## Simulate values.
def inv_logit(arr):
    return 1. / (1 + np.exp(-arr))

phi = norm(0,1).rvs(n_agents)
phi = inv_logit( a * ( phi + b ) )
psi = norm(mu,1).rvs(n_agents)

## Compute attentive response probabilities.
tau = norm(0,1).ppf([0.17,0.50,0.83])
theta = norm(0,1).cdf(np.subtract.outer(tau, psi)).T.round(6)
theta = np.column_stack([np.zeros(n_agents), theta, np.ones(n_agents)])
theta = np.diff(theta)

## Simulate responses.
Y = np.zeros((n_agents, n_items),dtype=int)

for i in range(n_items):
    
    ## Simulate attentive repsonse.
    y_effort = np.apply_along_axis(lambda p: np.random.multinomial(1, p), 1, theta)
    y_effort = np.argmax(y_effort, axis=1)
    
    ## Simulate random response.
    y_rand = np.random.randint(0,4,n_agents)
    
    ## Store response.
    w = np.random.binomial(1, phi)
    Y[:,i] = np.where(w, y_effort, y_rand)
    
## Compute sum scores.
scores = Y.sum(axis=1) / (3 * n_items)

## Initialize canvas.
fig, axes = plt.subplots(1,3,figsize=(15,4))

## Plot distribution of effortful responding.
sns.distplot(phi, kde=False, bins=np.linspace(0,1,11), ax=axes[0])
axes[0].set(xlabel='Trait Effort')

## Plot response likelihood.
axes[1].bar(np.arange(4), np.average(theta, axis=0, weights=phi))
axes[1].set(xticks=np.arange(4), )

## Plot distribution of sum scores.
sns.scatterplot(phi, scores, size=3, legend=False, ax=axes[2])
axes[2].set(xlabel='Trait Effort', ylabel='Sum Score')

sns.despine()
plt.tight_layout()

## Section XX: WSLS

In [ ]:
np.random.seed(47404)

def softmax(arr):
    """Scale-robust softmax choice rule."""
    arr = np.exp(arr - np.max(arr))
    return arr / arr.sum()

## Define metadata.
n_agents = 1000
n_trials = 90
n_arms = 3
epsilon = 0.05

## Simulate probabilities.
P = np.zeros((n_agents,n_arms))
P[np.arange(n_agents), np.argmax(np.random.normal(size=P.shape), axis=1)] = 1
P = np.where(P, 0.8, 0.2)

## Simulate rewards.
R = np.zeros((n_trials, n_agents, n_arms))
Q = np.zeros_like(R)

for i in range(n_trials):
    if not i % 15: P = np.apply_along_axis(np.roll, 1, P, np.random.choice([1,-1],1))
    Q[i] = P.copy()
    R[i] = np.random.binomial(1,P)
Q = np.argmax(Q, axis=-1)
    
    
## Simulate choice.
Y = np.zeros((n_trials,n_agents),dtype=int)
Y[0] = np.random.randint(0,3,(n_agents))

ix = np.arange(n_agents)

for i in range(1, n_trials):
    
    ## Prepopulate choice.
    p = np.zeros((n_agents,3))
    p[ix, Y[i-1]] = 1
    
    ## Observe previous outcome.
    r = R[i-1,ix,Y[i-1]]
    
    ## Update probabilities.
    p = np.abs(p.T - (1-r))
    p = (p / p.sum(axis=0)).T
    
    ## Add exploration.
    p = (1-epsilon) * p + (epsilon / 3)
    
    ## Simulate choice.
    f = lambda x: np.random.multinomial(1, x)
    Y[i] = np.argmax(np.apply_along_axis(f, 1, p), axis=1)
    
sns.distplot((Y == Q).mean(axis=0), bins=np.linspace(0.3,1,11))

# haha fuck you

In [ ]:
target = R.sum(axis=1).argmax(axis=1)

[np.mean(y==t) for y, t in zip(Y,target)]

In [ ]:
R

In [ ]:
R.shape